In [1]:
# 라이브러리 가져오기
import pandas as pd # Series, DataFrame 사용
import numpy as np # 값을 생성하는데 사용

### 계층 색인

- 행, 열의 각 축에 대해 다중 단계(계층)를 지정하여 데이터에 차원을 설정
- 인덱스에 다차원 리스트를 전달하면 계층 색인을 지정할 수 있음
- 데이터 구조를 재배열하거나 pivot 테이블과 같은 그룹 기반 작업에 유용
- 재배열 메서드
    - stack() : 컬럼을 로우로 피벗
    - unstack() : 로우를 컬럼으로 피벗

In [2]:
# Series
# 인덱스에 다차원 리스트(아이템 2개)를 전달
# 다차원리스트[0] 상위계층
# 상위계층 작성시 주의점 : 각 계층별로 속하는 하위계층 값의 개수만큼 계층명 작성
# 상위계층 리스트 개수 = 하위계층 리스트 개수
# 다차원리스트[1] : 하위계층
# 실습 --->
# 로우 인덱스 - 상위계층 : a, b, c, d
# 하위계층 : a(1, 2, 3), b(1, 2), c(1, 2, 3, 4), d(1)
s1 = pd.Series(np.arange(10), index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'c', 'c', 'd'],
                                    [1, 2, 3, 1, 2, 1, 2, 3, 4, 1]])
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [3]:
# 인덱스 확인
s1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 2),
            ('c', 1),
            ('c', 2),
            ('c', 3),
            ('c', 4),
            ('d', 1)],
           )

In [4]:
# 상위계층에 접근
# 계층 색인이 적용된 객체에 상위 인덱스에 접근 : 일반적인 Series 인덱싱으로 접근
s1['c']

1    5
2    6
3    7
4    8
dtype: int32

In [5]:
# 계층색인에 대한 슬라이싱 : 마지막 인덱스도 포함('b' : 'd' 로 입력시 b, c, d로 조회)
s1['b':'d']

b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [6]:
s1

a  1    0
   2    1
   3    2
b  1    3
   2    4
c  1    5
   2    6
   3    7
   4    8
d  1    9
dtype: int32

In [7]:
# 하위계층에 접근
# a, b, c, d에서 하위계층 라벨번호 2인 데이터만 조회
# 데이터[상위계층명, 하위계층번호]
s1[:, 2]

a    1
b    4
c    6
dtype: int32

In [9]:
# 하위 계층에 슬라이싱 : loc메서드를 이용해서 슬라이싱
# 상위계층이 c이고, 하위계층을 2~4까지 슬라이싱(.loc는 마지막 번호도 포함)
s1['c'].loc[2:4]

2    6
3    7
4    8
dtype: int32

In [10]:
# unstack() : 최하위(기본동작) 에 있는 로우 계층을 컬럼으로 적용하여 위로 올림
# Series 객체를 DataFrame화 할 수 있음
# NaN : 기존에 없던 로우 계층에 대한 값
s1.unstack()

,1,2,3,4
a,0.0,1.0,2.0,NaN
b,3.0,4.0,NaN,NaN
c,5.0,6.0,7.0,8.0
d,9.0,NaN,NaN,NaN


In [12]:
# stack() 메서드 : 컬럼에 있던 값을 가장 하위 인덱스로 내려서 재배열
# DataFrame을 다시 Series화 할 때 사용
s1.unstack().stack()

a  1    0.0
   2    1.0
   3    2.0
b  1    3.0
   2    4.0
c  1    5.0
   2    6.0
   3    7.0
   4    8.0
d  1    9.0
dtype: float64

In [16]:
# DataFrame 생성
# 구조 : 4 * 5
# 로우 인덱스 : 상위(2017, 2018) / 하위(모든 상위 인덱스에 대해 동일하게 a, b)
# 컬럼 인덱스 : 상위(서울, 경기) / 하위(서울-강남, 잠실 / 경기-안양, 수원, 판교)
# 값 : 1씩 증가하는 20개
df = pd.DataFrame(np.arange(20).reshape(4, 5), index=[[2017, 2017, 2018, 2018], ['a', 'b', 'a', 'b']],
            columns=[['서울', '서울', '경기', '경기', '경기'],['강남', '잠실', '안양', '수원', '판교']])
df

서울      경기        
        강남  잠실  안양  수원  판교
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

In [17]:
# 컬럼의 상위 계층 접근 : 일반적인 df컬럼 인덱싱 방식
# '서울' 데이터 조회
df['서울']

강남  잠실
2017 a   0   1
     b   5   6
2018 a  10  11
     b  15  16

In [19]:
# 컬럼이 이중으로 구조가 만들어져있으므로 인덱싱도 이중으로 해야 하나의 요소를 지정할 수 있음
df['서울']['강남']

2017  a     0
      b     5
2018  a    10
      b    15
Name: 강남, dtype: int32

In [21]:
# 튜플로 전달시 하나의 리스트로 2개 이상의 계층 처리 가능
df[('서울', '강남')]

2017  a     0
      b     5
2018  a    10
      b    15
Name: (서울, 강남), dtype: int32

In [23]:
# row의 상위 계층 접근
# 2017년 데이터를 조회해주세요.
df.loc[2017]

서울    경기      
  강남 잠실 안양 수원 판교
a  0  1  2  3  4
b  5  6  7  8  9

In [25]:
# 2017년도 전반기(a)데이터 조회
df.loc[2017].loc['a']

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: a, dtype: int32

In [26]:
# 위 조회구문에서 데이터를 튜플로 전달하기
df.loc[(2017, 'a')]

서울  강남    0
    잠실    1
경기  안양    2
    수원    3
    판교    4
Name: (2017, a), dtype: int32

In [30]:
# 안양 ~ 수워낚지 데이터 조회
# 주의점 : 데이터프레임 컬럼은 row 슬라이싱 결과에 대해서만 컬럼슬라이싱 적용 가능
#df['경기']['안양':'수원']
df['경기'].loc[:, "안양":"수원"]

안양  수원
2017 a   2   3
     b   7   8
2018 a  12  13
     b  17  18

In [31]:
# row의 최하위계층(a, b)를 컬럼으로 올리기 : unstack()
df.unstack()

서울              경기                    
      강남      잠실      안양      수원      판교    
       a   b   a   b   a   b   a   b   a   b
2017   0   5   1   6   2   7   3   8   4   9
2018  10  15  11  16  12  17  13  18  14  19

In [32]:
# 컬럼 최하위계층을 로우의 하위계층으로 재배열 : stack()
df.stack()

경기    서울
2017 a 강남   NaN   0.0
       수원   3.0   NaN
       안양   2.0   NaN
       잠실   NaN   1.0
       판교   4.0   NaN
     b 강남   NaN   5.0
       수원   8.0   NaN
       안양   7.0   NaN
       잠실   NaN   6.0
       판교   9.0   NaN
2018 a 강남   NaN  10.0
       수원  13.0   NaN
       안양  12.0   NaN
       잠실   NaN  11.0
       판교  14.0   NaN
     b 강남   NaN  15.0
       수원  18.0   NaN
       안양  17.0   NaN
       잠실   NaN  16.0
       판교  19.0   NaN

In [33]:
# 계층의 인덱스번호 또는 라벨을 사용하여 상, 하위간 교환
# swaplevel(key1, key2, axis=0(기본값))
# axis가 0인 경우는 row의 상하위 개념이 뒤집힌다.
df.swaplevel(1, 0)

서울      경기        
        강남  잠실  안양  수원  판교
a 2017   0   1   2   3   4
b 2017   5   6   7   8   9
a 2018  10  11  12  13  14
b 2018  15  16  17  18  19

In [34]:
# swaplevel에 axis=1을 주면 로우가 아닌 컬럼에 변동이 생깁니다.
df.swaplevel(1, 0, axis=1)

강남  잠실  안양  수원  판교
        서울  서울  경기  경기  경기
2017 a   0   1   2   3   4
     b   5   6   7   8   9
2018 a  10  11  12  13  14
     b  15  16  17  18  19

### 객체 복사

- 할당기호(=) / 슬라이싱 : 원본과 상호 종속적인 복사본 객체 생성(얕은복사)
- obj.copy() : 원본과 독립적인 복사본 객체 생성(깊은복사)

In [35]:
# 모든 값이 1인 5 * 1 구조의 Series 생성
s1 = pd.Series(np.ones(5))
s1

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
dtype: float64

In [37]:
# 할당기호 = 로 복사 : 얕은복사
s2 = s1

In [38]:
# 할당기호로 복사한 객체의 아이템 수정
s2[0] = 10
s2

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [39]:
# = 를 이용한 대입은 원본객체도 동시에 변경됨
s1

0    10.0
1     1.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [40]:
# 원본객체의 아이템 수정
s1[1] = 3
s1

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [41]:
# 복사본도 영향받음
s2

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [42]:
# 슬라이싱으로 복사(일반 파이썬 -> 깊은복사, pandas-> 얕은복사로 취급됨)
s3 = s1[:]
s3

0    10.0
1     3.0
2     1.0
3     1.0
4     1.0
dtype: float64

In [43]:
# 복사본 객체 아이템 수정
s3[2] = -9
s3

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [46]:
# s1과 s3 확인
print(s1)
print("------------")
print(s3)

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64
------------
0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64


In [47]:
# .copy()를 이용한 깊은복사(복사)
s4 = s1.copy()

In [48]:
# s4를 수정
s4[4] = 44

In [49]:
s4

0    10.0
1     3.0
2    -9.0
3     1.0
4    44.0
dtype: float64

In [50]:
s1

0    10.0
1     3.0
2    -9.0
3     1.0
4     1.0
dtype: float64

In [51]:
# s1을 변경하는 경우에도 s4는 영향받지 않음
s1[0] = 0

In [53]:
s1

0    0.0
1    3.0
2   -9.0
3    1.0
4    1.0
dtype: float64

In [54]:
s4

0    10.0
1     3.0
2    -9.0
3     1.0
4    44.0
dtype: float64

### 정렬

- obj.sort_index() : 인덱스를 기준으로 정렬 (기본값은 ascending=True, 오름차순 정렬)
    - DataFrame, Series
        - axis = 0 : 기본값, 로우 인덱스 기준으로 정렬
        - axis = 1 : 컬럼 인덱스 기준으로 정렬
- obj.sort_values() : 값을 기준으로 정렬
    - DataFrame, Series
        - by : 정렬의 기준이 되는 인덱스 값 전달
        - axis = 0 : 기본값, 컬럼을 기준으로 로우 인덱스를 정렬하며 기준값으로 by에 인덱스 컬럼 레벨 또는 컬럼명 전달
        - axis = 1 : 로우 인덱스를 기준으로 컬럼 라벨을 정렬하며 기준값으로 by에 레벨 또는 라벨명 전달

In [55]:
# Series 생성
# 값과 인덱스라벨이 순서대로 들어가지 않은 Series
s1 = pd.Series([2, 3, 1, 7, 0], index=list('gacfd'))
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [56]:
# 인덱스 기준 오름차순으로 정렬
# 기본동작 : 오름차순 & 로우 인덱스
s1.sort_index()

a    3
c    1
d    0
f    7
g    2
dtype: int64

In [57]:
# 인덱스 기준 내림차순으로 정렬
s1.sort_index(ascending=False)

g    2
f    7
d    0
c    1
a    3
dtype: int64

In [58]:
# 일시적 변경이므로 s1에 반영되지는 않습니다.
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [59]:
# 값 기준 오름차순으로 정렬
s1.sort_values()

d    0
c    1
g    2
a    3
f    7
dtype: int64

In [60]:
# 값 기준 내림차순으로 정렬
s1.sort_values(ascending=False)

f    7
a    3
g    2
c    1
d    0
dtype: int64

In [61]:
# 역시 원본에 반영되진 않습니다.
s1

g    2
a    3
c    1
f    7
d    0
dtype: int64

In [68]:
np.random.seed(1234)

In [69]:
# DataFrame 생성
# 4 * 5, 무작위 정수(np.random.randint())
# 로우/컬럼 인덱스도 순서가 없는 값 지정
# 로우 'hcae', 컬럼은 'EAFCD'
df1 = pd.DataFrame(np.random.randint(20, size=(4, 5)), index=list('hcae'), columns=list('EAFCD'))
df1

,E,A,F,C,D
h,15,19,6,12,15
c,17,9,11,12,16
a,5,16,9,15,18
e,16,12,5,2,6


In [71]:
# 로우 인덱스 기준으로 오름차순 정렬(axis=0)
df1.sort_index()

,E,A,F,C,D
a,5,16,9,15,18
c,17,9,11,12,16
e,16,12,5,2,6
h,15,19,6,12,15


In [72]:
# 로우 인덱스 기준으로 내림차순 정렬(axis=0)
df1.sort_index(ascending=False)

,E,A,F,C,D
h,15,19,6,12,15
e,16,12,5,2,6
c,17,9,11,12,16
a,5,16,9,15,18


In [73]:
# 컬럼명 기준으로 오름차순 정렬(axis=1)
df1.sort_index(axis=1)

,A,C,D,E,F
h,19,12,15,15,6
c,9,12,16,17,11
a,16,15,18,5,9
e,12,2,6,16,5


In [74]:
# 컬럼명 기준으로 내림차순 정렬
df1.sort_index(axis=1, ascending=False)

,F,E,D,C,A
h,6,15,15,12,19
c,11,17,16,12,9
a,9,5,18,15,16
e,5,16,6,2,12


In [75]:
# 컬럼 기준으로 내림차순 정렬 후, 로우 기준으로 오름차순 정렬
df1.sort_index(axis=1, ascending=False).sort_index()

,F,E,D,C,A
a,9,5,18,15,16
c,11,17,16,12,9
e,5,16,6,2,12
h,6,15,15,12,19


In [76]:
# 정렬 결과가 원본에는 반영이 안 됨
df1

,E,A,F,C,D
h,15,19,6,12,15
c,17,9,11,12,16
a,5,16,9,15,18
e,16,12,5,2,6


In [77]:
# 값 기준으로 정렬
# 컬럼 D의 값을 기준으로 오름차순 정렬
# sort_values(axis=0, by='기준컬럼명') : 기본동작(정렬결과를 로우에 반영)
df1.sort_values(by='D', axis=0)

,E,A,F,C,D
e,16,12,5,2,6
h,15,19,6,12,15
c,17,9,11,12,16
a,5,16,9,15,18


In [78]:
# 컬럼 A를 기준으로 내림차순 정렬
df1.sort_values(by='A', ascending=False)

,E,A,F,C,D
h,15,19,6,12,15
a,5,16,9,15,18
e,16,12,5,2,6
c,17,9,11,12,16


In [80]:
# 인덱스라벨(row) c의 값을 오름차순으로 정렬
# 결과적으로 정렬되는 대상 : 컬럼
# 정렬의 기준 : row 라벨
df1.sort_values(by='c', axis=1)

,A,F,C,D,E
h,19,6,12,15,15
c,9,11,12,16,17
a,16,9,15,18,5
e,12,5,2,6,16


In [81]:
# 로우 'e'를 기준으로 값을 내림차순으로 정렬
df1.sort_values(by='e', axis=1, ascending=False)

,E,A,D,F,C
h,15,19,15,6,12
c,17,9,16,11,12
a,5,16,18,9,15
e,16,12,6,5,2


In [83]:
# 두 개의 컬럼에 대해서 정렬 : 리스트로 묶어서 by의 인자값으로 전달
# 정렬 우선순위 : 차례대로 1순위 > 2순위(1순위 동점시 2순위로 넘어감)
# 1순위 정렬 후 1순위 컬럼의 동점값에 대해 2순위 컬럼의 값이 재정렬
# 동일한 값이 아니라면 무조건 1순위 컬럼이 우선
df1.sort_values(by=['C', 'A'])

,E,A,F,C,D
e,16,12,5,2,6
c,17,9,11,12,16
h,15,19,6,12,15
a,5,16,9,15,18


In [84]:
# C는 오름차순 A는 내림차순으로 하고 싶다면
# ascending = [True, False] 와 같이 각각의 컬럼에 대해서 ascending을 따로 매겨준다
df1.sort_values(by=['C', 'A'], ascending=[True, False])

,E,A,F,C,D
e,16,12,5,2,6
h,15,19,6,12,15
c,17,9,11,12,16
a,5,16,9,15,18


### 연습문제

#### 1. 아래의 데이터프레임을 생성하세요

- 학생들의 점수는 50 이상 100 미만의 무작위 정수 값을 생성하여 사용

<img src="img/df_sort_practice1.png" width=250 align='left'>

In [85]:
# 데이터프레임 생성
df = pd.DataFrame(np.random.randint(50, 100, 20).reshape(5, 4),
                 index=['Kim', 'Park', 'Lee', 'Jung', "Moon"],
                 columns=[[2016, 2016, 2017, 2017], ['영어', '수학', '영어', '수학']])
df

2016     2017    
       영어  수학   영어  수학
Kim    53  89   61  50
Park   91  61   66  53
Lee    52  69   62  51
Jung   61  93   67  64
Moon   57  92   93  96

In [87]:
# 컬럼 인덱스 이름 설정(리스트로 넘김)
# 멀티 인덱스는 무조건 set_names를 이용해야 입력 가능
df.columns.set_names(['연도', '과목명'], inplace=True)

In [88]:
df

연도   2016     2017    
과목명    영어  수학   영어  수학
Kim    53  89   61  50
Park   91  61   66  53
Lee    52  69   62  51
Jung   61  93   67  64
Moon   57  92   93  96

In [89]:
# 로우 인덱스 '학생명' 으로 설정
df.index.name = '학생명'
df

연도   2016     2017    
과목명    영어  수학   영어  수학
학생명                   
Kim    53  89   61  50
Park   91  61   66  53
Lee    52  69   62  51
Jung   61  93   67  64
Moon   57  92   93  96

#### 2. 2016년 데이터만 별도의 데이터프레임으로 분리 저장하세요.(깊은복사)

In [91]:
df2016 = df[2016].copy()
df2016

과목명,영어,수학
학생명,,
Kim,53,89
Park,91,61
Lee,52,69
Jung,61,93
Moon,57,92


#### 3. 2016년 데이터에 대해 학생 이름을 기준으로 오름차순 정렬하세요.

In [93]:
df2016.sort_index()

과목명,영어,수학
학생명,,
Jung,61,93
Kim,53,89
Lee,52,69
Moon,57,92
Park,91,61


#### 4. 2016년 데이터의 과목별 점수에 대하여 영어 점수가 높은 순서로 정렬하고, 영어 점수 동점자의 경우 수학 점수가 낮은 순서로 정렬하세요.

In [94]:
df2016.sort_values(by=['영어', '수학'], ascending=[False, True])

과목명,영어,수학
학생명,,
Park,91,61
Jung,61,93
Moon,57,92
Kim,53,89
Lee,52,69


### 데이터 합치기

- merge
    - pandas 객체의 메서드로 pandas.merge(df1, df2 ...)로 사용
    - 두 개의 데이터프레임에 대해 특정 컬럼을 기준으로 합치기
    - 주요 파라미터
        - how : 합치는 방식으로 inner(기본값), left, right, outer 방식 존재
        - on : 합치는 기준으로 두 개의 데이터프레임에 공통으로 존재하는 컬럼명을 사용해야함 (기본값=None)

In [95]:
# 고객번호와 고객명을 저장할고 있는 데이터프레임 생성
# 딕셔너리 타입으로 데이터프레임 데이터 생성 : key:col, value-> list() : row
df1 = pd.DataFrame({'no':[30, 31, 32, 33, 34],
                    'name':['김파이썬', '이장고', '박팬더스', '강넘파이', '최주피터']})
df1

,no,name
0,30,김파이썬
1,31,이장고
2,32,박팬더스
3,33,강넘파이
4,34,최주피터


In [96]:
# 고객 번호와 주문수량을 저장하고 있는 데이터프레임 생성
# 다른 고객번호(no)를 일부 지정
df2 = pd.DataFrame({'no':[30, 33, 40, 41], 
                   'amount':[100, 130, 40, 60]})
df2

,no,amount
0,30,100
1,33,130
2,40,40
3,41,60


In [97]:
# 가장 기본적인 merge 방식 : 공통 컬럼을 기준으로 두 df에 모두 존재하는 교집합만 추출
# 기본 동작 : how = 'inner'
# df1 & df2의 공통 컬럼 : no
# 공통 컬럼 내부의 공통 값 : 30, 33
# df1 : 30 ~ 34
# df2 : 30, 33, 40, 41
pd.merge(df1, df2)

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130


In [98]:
# how= 'outer' : 결합 기준으로 공통컬럼을 사용함
# 교집합이 아닌 부분애 대해서는 NaN으로 처리(누락되는 데이터는 없음)
pd.merge(df1, df2, how='outer')

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN
5,40,NaN,40.0
6,41,NaN,60.0


In [99]:
# left : 첫 번째로 전달한 df1의 데이터는 모두 살리고, df2에서는 df1과 겹칠때만 남김
pd.merge(df1, df2, how='left')

,no,name,amount
0,30,김파이썬,100.0
1,31,이장고,NaN
2,32,박팬더스,NaN
3,33,강넘파이,130.0
4,34,최주피터,NaN


In [100]:
# right : 두 번째 데이터프레임의 데이터를 보존
# df1에서는 df2와 겹치는 경우에 한해 데이터를 남김
pd.merge(df1, df2, how='right')

,no,name,amount
0,30,김파이썬,100
1,33,강넘파이,130
2,40,NaN,40
3,41,NaN,60


In [101]:
# 공통 컬럼이 두 개 이상인 경우
# '고객명', '날짜', '정보' 커럼에 3개의 데이터를 저장하고 있는 데이터프레임 생성
df3 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                   '날짜':['2022-08-01', '2022-08-02', '2022-08-03'],
                   '정보':['010', '011', '019']})
df3

,고객명,날짜,정보
0,김파이썬,2022-08-01,010
1,이장고,2022-08-02,011
2,박팬더스,2022-08-03,019


In [102]:
# 고객명, 정보를 저장하고 있는 데이터프레임 생성
df4= pd.DataFrame({'고객명':['김파이썬', '박팬더스', '최넘파이'],
                  '정보':['F', 'M', 'M']})
df4

,고객명,정보
0,김파이썬,F
1,박팬더스,M
2,최넘파이,M


In [103]:
# 공통된 이름을 가진 컬럼이 2개 이상인 경우, 파라미터가 없으면 결과도 없다.
pd.merge(df3, df4)

,고객명,날짜,정보


In [106]:
# on 파라미터 : 공통된 컬럼이 여럿인 경우 결합 기준 컬럼을 지정
# 합칠때 key로 사용할 컬럼 : '고객명'
# 공통컬럼 결과 : 고객명, 정보 -> 정보 컬럼 결과를 확인
# 기본동작으로 merge : inner
pd.merge(df3, df4, on='고객명')

,고객명,날짜,정보_x,정보_y
0,김파이썬,2022-08-01,010,F
1,박팬더스,2022-08-03,019,M


In [107]:
# left_on, right_on : 두 개의 데이터프레임에 대해서 서로 다른 기준컬럼을 지정
# 예) 동일한 속성의 자료를 저장하는 컬럼인데 이름이 다른 경우
# 고객이름, 날짜, 구매금액을 저장하고 있는 데이터프레임 생성
df5 = pd.DataFrame({'고객이름':['김파이썬', '박팬더스', '강주피터'],
                   '날짜':['2022-08-01', '2022-08-02', '2022-08-03'],
                   '구매금액':[1, 2, 3]})
df5

,고객이름,날짜,구매금액
0,김파이썬,2022-08-01,1
1,박팬더스,2022-08-02,2
2,강주피터,2022-08-03,3


In [108]:
# 고객명, 성별을 지정하고 있는 데이터프레임을 생성
df6 = pd.DataFrame({'고객명':['김파이썬', '박팬더스'],
                   '성별':['F', 'M']})
df6

,고객명,성별
0,김파이썬,F
1,박팬더스,M


In [109]:
# 겹치는 컬럼이 없어서 에러 발생
pd.merge(df5, df6)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [111]:
# left(df5) right(df6) 공통컬럼명을 left_on='컬럼명1', right_on='컬럼명2' 형식으로 지정해주요.
tmp = pd.merge(df5, df6, left_on='고객이름', right_on='고객명')
tmp

,고객이름,날짜,구매금액,고객명,성별
0,김파이썬,2022-08-01,1,김파이썬,F
1,박팬더스,2022-08-02,2,박팬더스,M


In [112]:
# drop 메서드를 이용해 '고객명' 컬럼을 삭제해주세요.
tmp.drop('고객명', axis=1, inplace=True)
#tmp.drop(columns='고객명', inplace=True)
tmp

,고객이름,날짜,구매금액,성별
0,김파이썬,2022-08-01,1,F
1,박팬더스,2022-08-02,2,M


#### concat

- 특정 key를 기준으로 데이터를 합치는 것이 아니라 행, 열 기준으로 데이터를 연결
- 주요 파라미터
    - axis : 0 / 행 방향(기본값)이며 컬럼을 key로 합치고, 1 / 열 방향으로 로우를 key로 합침
    - join : 데이터프레임끼리 연결할 때 합치는 방법으로 outer(기본값), inner 방식 존재
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [113]:
# 공통 인덱스 라벨을 가지는 Series 2개 생성
s1 = pd.Series([1, 2, 3], index=list('abc'))
s2 = pd.Series([5, 6, 7, 8], index=list('abfh'))
print(s1)
print(s2)

a    1
b    2
c    3
dtype: int64
a    5
b    6
f    7
h    8
dtype: int64


In [114]:
# 두 Series간 연결
# 기본 : axis=0(행 방향으로 연결)
# 첫 번째로 전달된 객체가 위에, 두 번째로 전달된 객체가 아래로 추가(연결)
# 인덱스 라벨은 기존 값 유지
pd.concat([s1, s2])

a    1
b    2
c    3
a    5
b    6
f    7
h    8
dtype: int64

In [115]:
# 붙이면서 새로운 인덱스로 초기화하기
pd.concat([s1, s2], ignore_index=True)

0    1
1    2
2    3
3    5
4    6
5    7
6    8
dtype: int64

In [116]:
# 열 방향 연결 -> 두 개의 Series를 연결해서 하나의 DataFrame으로 만들기
# 길이가 다른 경우
pd.concat([s1, s2], axis=1)

,0,1
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [118]:
# 열 이름을 설정하면서 붙여주기 : keys 파라미터에 컬럼명을 리스트로 전달
pd.concat([s1, s2], axis=1, keys=['c1', 'c2'])

,c1,c2
a,1.0,5.0
b,2.0,6.0
c,3.0,NaN
f,NaN,7.0
h,NaN,8.0


In [119]:
# 두 개의 데이터프레임 연결
# 고객명, 날짜, 금액
df1 = pd.DataFrame({'고객명':['김파이썬', '이장고', '박팬더스'],
                  '날짜':['2022-08-01', '2022-08-02', '2022-08-03'],
                  '구매금액':[1, 2, 3]})
# 고객명, 성별
df2 = pd.DataFrame({'고객명':['김파이썬', '최넘파이'],
                   '성별':['F', 'M']})

In [120]:
df1

,고객명,날짜,구매금액
0,김파이썬,2022-08-01,1
1,이장고,2022-08-02,2
2,박팬더스,2022-08-03,3


In [121]:
df2

,고객명,성별
0,김파이썬,F
1,최넘파이,M


In [122]:
# 행 방향으로(row개수 증가) 데이터프레임 연결
pd.concat([df1, df2])

,고객명,날짜,구매금액,성별
0,김파이썬,2022-08-01,1.0,NaN
1,이장고,2022-08-02,2.0,NaN
2,박팬더스,2022-08-03,3.0,NaN
0,김파이썬,NaN,NaN,F
1,최넘파이,NaN,NaN,M


In [123]:
# 열 방향으로 데이터프레임 연결 
pd.concat([df1, df2], axis=1)

,고객명,날짜,구매금액,고객명,성별
0,김파이썬,2022-08-01,1,김파이썬,F
1,이장고,2022-08-02,2,최넘파이,M
2,박팬더스,2022-08-03,3,NaN,NaN


In [124]:
df3 = pd.DataFrame({'지역':['서울', '부산', '제주'],
                   '직업':['웹개발자', '회계사', '회사원']})
df3

,지역,직업
0,서울,웹개발자
1,부산,회계사
2,제주,회사원


In [125]:
# 전제조건 : df1과 df3의 데이터가 순서, 맥락상 맞는것이 확정적일때 concat으로 합치면 좋습니다.
pd.concat([df1, df3], axis=1)

,고객명,날짜,구매금액,지역,직업
0,김파이썬,2022-08-01,1,서울,웹개발자
1,이장고,2022-08-02,2,부산,회계사
2,박팬더스,2022-08-03,3,제주,회사원


### append
- 행 방향으로 데이터를 연결
- 주요 파라미터
    - ignore_index : 합친 후 기존 인덱스를 유지 또는 새로운 인덱스를 지정

In [126]:
# 두 개의 시리즈 생성
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6, 7])

In [127]:
# 행 방향으로 데이터 연결하고 기존 인덱스 유지
s1.append(s2)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\3431566202.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s1.append(s2)


0    1
1    2
2    3
0    4
1    5
2    6
3    7
dtype: int64

In [128]:
# 행 방향으로 데이터 연결하고 기존 인덱스 누락시키기
s1.append(s2, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\4246903571.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s1.append(s2, ignore_index=True)


0    1
1    2
2    3
3    4
4    5
5    6
6    7
dtype: int64

In [129]:
# 두 개의 데이터프레임
# 서로 다른 컬럼명
df1 = pd.DataFrame([[1, 2], 
                    [3, 4]], columns=list('ab'))
df2 = pd.DataFrame([[5, 6], 
                    [7, 8], 
                    [9, 10]], columns=list('AB'))
print(df1)
print(df2)

   a  b
0  1  2
1  3  4
   A   B
0  5   6
1  7   8
2  9  10


In [130]:
df1.append(df2)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\3062608662.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df2)


,a,b,A,B
0,1.0,2.0,NaN,NaN
1,3.0,4.0,NaN,NaN
0,NaN,NaN,5.0,6.0
1,NaN,NaN,7.0,8.0
2,NaN,NaN,9.0,10.0


In [131]:
# 두 개의 데이터프레임
# 동일한 컬럼명
df3 = pd.DataFrame([[10, 20],
                   [30, 40],
                   [50, 60]], columns=list('ab'))
df3

,a,b
0,10,20
1,30,40
2,50,60


In [132]:
# 동일한 컬럼명에 대해서 append를 하면 컬럼갯수는 원본 데이터 숫자 그대로, 행만 추가
# append 는 보통 이런 케이스에 많이 사용함
df1.append(df3)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\364716140.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df3)


,a,b
0,1,2
1,3,4
0,10,20
1,30,40
2,50,60


In [133]:
# ignore_index를 걸면 인덱스번호도 0부터 다시 초기화
df1.append(df3, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\1777171504.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(df3, ignore_index=True)


,a,b
0,1,2
1,3,4
2,10,20
3,30,40
4,50,60


In [134]:
# 시리즈와 데이터프레임
# 데이터프레임에 한 줄 추가할때 보통 시리즈를 많이 사용함
# 시리즈 라벨 & 데이터프레임 컬럼명이 서로 일치할때 매칭됨
# 시리즈 라벨, 데이터프레임 컬럼명을 모두 a,b,c,d로 매칭시킴
s1 = pd.Series([10, 20, 30, 40], index=list('abcd'))
df1 = pd.DataFrame([[1, 1, 1, 1],
                   [2, 2, 2, 2]], columns=list('abcd'))

In [135]:
s1

a    10
b    20
c    30
d    40
dtype: int64

In [136]:
df1

,a,b,c,d
0,1,1,1,1
1,2,2,2,2


In [137]:
df1.append(s1, ignore_index=True)

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_56756\3381292091.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1.append(s1, ignore_index=True)


,a,b,c,d
0,1,1,1,1
1,2,2,2,2
2,10,20,30,40
